In [1]:
import pandas as pd


In [ ]:
dados = pd.read_parquet("Dados/pl_fundos.parquet")
print(dados)
print(dados.dtypes)


     Fundos/Carteiras Adm              PL
0             FIRF GERAES  R$ 487.653.973
1          FIRF GERAES 30  R$ 334.283.724
2            BORDEAUX FIM    R$ 1.803.463
3                  BH FIM      R$ 585.692
4            GLOBAL BONDS   R$ 30.863.794
5             TOPAZIO FIM    R$ 2.108.555
6               AF TÁTICO   R$ 50.245.681
7      ROMEU FC FIM CP IE  R$ 233.654.393
8               HORIZONTE  R$ 331.380.996
9                JERA2026  R$ 208.959.123
10               REAL FIM   R$ 39.865.977
11           SANKALPA FIM      R$ 615.856
12            AYA NMK FIM      R$ 592.084
13  AF INVEST GERAES PREV      R$ 792.822
14    BBRASIL FIM CP RESP  R$ 127.865.818
15                BMG SEG  R$ 125.891.320
16          BH FIRF INFRA  R$ 208.056.498
17         BORDEAUX INFRA   R$ 50.717.917
18          TOPAZIO INFRA   R$ 35.885.760
19      MANACA INFRA FIRF   R$ 44.659.017
20              RIO INFRA            R$ 0
22    AF DEB INCENTIVADAS   R$ 12.864.022
23                  IMA-B         

In [6]:
import pandas as pd
import re

# 1. Simulando seu DataFrame original (do arquivo parquet)
data_original = {
    'Fundos/Carteiras Adm': [
        'FIRF GERAES', 'FIRF GERAES 30', 'BORDEAUX FIM', 'BH FIM', 'GLOBAL BONDS', 
        'TOPAZIO FIM', 'AF TÁTICO', 'ROMEU FC FIM CP IE', 'HORIZONTE', 'JERA2026', 
        'REAL FIM', 'SANKALPA FIM', 'AYA NMK FIM', 'AF INVEST GERAES PREV', 
        'BBRASIL FIM CP RESP', 'BMG SEG', 'BH FIRF INFRA', 'BORDEAUX INFRA', 
        'TOPAZIO INFRA', 'MANACA INFRA FIRF', 'RIO INFRA', 'AF DEB INCENTIVADAS', 'IMA-B'
    ],
    'PL': [
        'R$ 487.653.973', 'R$ 334.283.724', 'R$ 1.803.463', 'R$ 585.692', 'R$ 30.863.794',
        'R$ 2.108.555', 'R$ 50.245.681', 'R$ 233.654.393', 'R$ 331.380.996', 'R$ 208.959.123',
        'R$ 39.865.977', 'R$ 615.856', 'R$ 592.084', 'R$ 792.822', 'R$ 127.865.818', 
        'R$ 125.891.320', 'R$ 208.056.498', 'R$ 50.717.917', 'R$ 35.885.760', 'R$ 44.659.017',
        'R$ 0', 'R$ 12.864.022', '--'
    ]
}
dados = pd.DataFrame(data_original)
# Na prática você usaria: dados = pd.read_parquet("Dados/pl_fundos.parquet")

# 2. O Texto com os novos dados
texto_novos_dados = """
AF TÁTICO   R$ 50.318.975   --  -3.92% ...
BBRASIL FIM CP RESP R$ 94.658.558   -- ...
BH INFRA    R$ 216.294.329  -- ...
BMG SEG R$ 130.498.526  -- ...
BORDEAUX INFRA  R$ 52.871.909   -- ...
FIRF GERAES R$ 627.156.984  -- ...
FIRF GERAES 30  R$ 363.381.253  -- ...
GLOBAL BONDS    R$ 31.099.980   0.03% ...
HORIZONTE FIM   R$ 223.971.639  0.11% ...
JERA2026 FIM    R$ 216.947.276  -- ...
MANACA INFRA    R$ 46.451.211   -- ...
REAL FIM    R$ 41.307.013   -- ...
ROMEU FIM   R$ 241.791.999  -- ...
TOPAZIO INFRA   R$ 37.357.994   -- ...
AF DEB INCENTIVADAS R$ 1.230.315    0.24% ...
"""

# 3. Processamento: Extrair Nome e PL do texto bagunçado
# O regex procura pelo padrão "R$ numero.numero" e pega o que vem antes como nome
padrao_pl = re.compile(r"(.*?)\s+(R\$\s[\d\.]+)")
novos_valores = {}

for linha in texto_novos_dados.strip().split('\n'):
    match = padrao_pl.search(linha.strip())
    if match:
        nome_fundo = match.group(1).strip()
        valor_pl = match.group(2).strip()
        novos_valores[nome_fundo] = valor_pl

# 4. Mapeamento (De-Para)
# Como os nomes no arquivo original são diferentes dos nomes no texto novo,
# precisamos "traduzir" as chaves para o Python encontrar os valores.
# Chave = Nome no Parquet (Original) : Valor = Nome no Texto Novo
de_para = {
    'BH FIRF INFRA': 'BH INFRA',
    'ROMEU FC FIM CP IE': 'ROMEU FIM',
    'HORIZONTE': 'HORIZONTE FIM',
    'JERA2026': 'JERA2026 FIM',
    'MANACA INFRA FIRF': 'MANACA INFRA',
    # Os nomes abaixo são iguais, mas garantimos o mapeamento direto
    'AF TÁTICO': 'AF TÁTICO',
    'FIRF GERAES': 'FIRF GERAES',
    'FIRF GERAES 30': 'FIRF GERAES 30',
    'GLOBAL BONDS': 'GLOBAL BONDS',
    'REAL FIM': 'REAL FIM',
    'BBRASIL FIM CP RESP': 'BBRASIL FIM CP RESP',
    'BMG SEG': 'BMG SEG',
    'BORDEAUX INFRA': 'BORDEAUX INFRA',
    'TOPAZIO INFRA': 'TOPAZIO INFRA',
    'AF DEB INCENTIVADAS': 'AF DEB INCENTIVADAS'
}

# 5. Função para atualizar linha a linha
def atualizar_pl(row):
    nome_antigo = row['Fundos/Carteiras Adm']
    
    # Verifica qual é o nome correspondente na lista nova
    nome_novo_correspondente = de_para.get(nome_antigo, nome_antigo)
    
    # Se existir um valor novo para esse nome, atualiza. Se não, mantém o antigo.
    if nome_novo_correspondente in novos_valores:
        return novos_valores[nome_novo_correspondente]
    else:
        return row['PL']

# Aplicar a atualização
dados['PL'] = dados.apply(atualizar_pl, axis=1)

# 6. Exibir resultado
print(dados)
print("\nTipos dos dados:")
print(dados.dtypes)

# Para salvar:
dados.to_parquet("Dados/pl_fundos.parquet")

     Fundos/Carteiras Adm              PL
0             FIRF GERAES  R$ 627.156.984
1          FIRF GERAES 30  R$ 363.381.253
2            BORDEAUX FIM    R$ 1.803.463
3                  BH FIM      R$ 585.692
4            GLOBAL BONDS   R$ 31.099.980
5             TOPAZIO FIM    R$ 2.108.555
6               AF TÁTICO   R$ 50.318.975
7      ROMEU FC FIM CP IE  R$ 241.791.999
8               HORIZONTE  R$ 223.971.639
9                JERA2026  R$ 216.947.276
10               REAL FIM   R$ 41.307.013
11           SANKALPA FIM      R$ 615.856
12            AYA NMK FIM      R$ 592.084
13  AF INVEST GERAES PREV      R$ 792.822
14    BBRASIL FIM CP RESP   R$ 94.658.558
15                BMG SEG  R$ 130.498.526
16          BH FIRF INFRA  R$ 216.294.329
17         BORDEAUX INFRA   R$ 52.871.909
18          TOPAZIO INFRA   R$ 37.357.994
19      MANACA INFRA FIRF   R$ 46.451.211
20              RIO INFRA            R$ 0
21    AF DEB INCENTIVADAS    R$ 1.230.315
22                  IMA-B         